# Seq2Seq

이름 : 정성오

기수 : 10기

작성자 : 10 기 신재우

In [1]:
# colab 환경에서 학습을 진행하실 분들은 구글드라이브를 연동해주세요
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 시작 전 실행할 것들

In [2]:
%pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 45.3 MB/s eta 0:00:00


In [3]:
from konlpy.tag import Kkma
from konlpy.utils import pprint
import pandas as pd
import numpy as np
import tqdm
import spacy
import torchtext
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
import random
import os
from ast import literal_eval
import torchtext.vocab as vocab
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

### 경로 설정

In [4]:
path_to_folder = "/content/drive/MyDrive/[0225]RNN + Transformers_SungOhJung"

In [5]:
path_train = os.path.join(path_to_folder, "Dataset/train_dataset_Tokenized.csv")
path_val = os.path.join(path_to_folder, "Dataset/val_dataset_Tokenized.csv")
path_test = os.path.join(path_to_folder, "Dataset/test_dataset_Tokenized.csv")
train_dataset = pd.read_csv(path_train, index_col = 0)
val_dataset = pd.read_csv(path_val, index_col = 0)
test_dataset = pd.read_csv(path_test, index_col = 0)

# Vocabulary

In [6]:
train_dataset['en_tokens'] = train_dataset['en_tokens'].apply(literal_eval)
train_dataset['kr_tokens'] = train_dataset['kr_tokens'].apply(literal_eval)

val_dataset['en_tokens'] = val_dataset['en_tokens'].apply(literal_eval)
val_dataset['kr_tokens'] = val_dataset['kr_tokens'].apply(literal_eval)

test_dataset['en_tokens'] = test_dataset['en_tokens'].apply(literal_eval)
test_dataset['kr_tokens'] = test_dataset['kr_tokens'].apply(literal_eval)

train_dict = train_dataset.to_dict(orient = 'records')
val_dict = val_dataset.to_dict(orient = 'records')
test_dict = test_dataset.to_dict(orient = 'records')

In [7]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"
sos_token = "<sos>"
eos_token = "<eos>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

In [8]:
def yield_tokens(data, token_field):
    for item in data:
        yield item[token_field]

en_vocab = vocab.build_vocab_from_iterator(
    yield_tokens(train_dict, "en_tokens"),
    specials=special_tokens,
    min_freq=min_freq
)

kr_vocab = vocab.build_vocab_from_iterator(
    yield_tokens(train_dict, "kr_tokens"),
    specials=special_tokens,
    min_freq=min_freq
)

en_vocab.set_default_index(en_vocab[unk_token])
kr_vocab.set_default_index(kr_vocab[unk_token])

In [9]:
vocab_size_en = len(en_vocab)
vocab_size_kr = len(kr_vocab)
print("English Vocabulary Length : ", vocab_size_en)
print("Korean Vocabulary Length : ", vocab_size_kr)

English Vocabulary Length :  9772
Korean Vocabulary Length :  10442


# 문제 2 (Vocabulary Matching)

토큰화된 단어들을 Vocabulary 과 매칭 시켜서 **벡터**로 바꿔준 뒤에 **텐서**로 바꿔주는 단계까지 포함되어야 합니다!

힌트 :

- $\texttt{en_vocab.lookup_indices}$, 혹은 $\texttt{kr_vocab.lookup_indices}$
- 한줄로 표현이 가능합니다!
- en_indices 와 kr_indices 는 리스트이며, 텐서화 된 것들을 각 토큰마다 리스트로 가져야 합니다!

In [10]:
en_indices = []
for word in train_dataset['en_tokens']:
  en_indices.append(torch.LongTensor(𝚎𝚗_𝚟𝚘𝚌𝚊𝚋.𝚕𝚘𝚘𝚔𝚞𝚙_𝚒𝚗𝚍𝚒𝚌𝚎𝚜(word)))
train_dataset['en_indices'] = en_indices

kr_indices = []
for word in train_dataset['kr_tokens']:
  kr_indices.append(torch.LongTensor(𝚔𝚛_𝚟𝚘𝚌𝚊𝚋.𝚕𝚘𝚘𝚔𝚞𝚙_𝚒𝚗𝚍𝚒𝚌𝚎𝚜(word)))

train_dataset['kr_indices'] = kr_indices

In [11]:
en_indices = []
for word in val_dataset['en_tokens']:
  en_indices.append(torch.LongTensor(𝚎𝚗_𝚟𝚘𝚌𝚊𝚋.𝚕𝚘𝚘𝚔𝚞𝚙_𝚒𝚗𝚍𝚒𝚌𝚎𝚜(word)))

val_dataset['en_indices'] = en_indices

kr_indices = []
for word in val_dataset['kr_tokens']:
  kr_indices.append(torch.LongTensor(𝚔𝚛_𝚟𝚘𝚌𝚊𝚋.𝚕𝚘𝚘𝚔𝚞𝚙_𝚒𝚗𝚍𝚒𝚌𝚎𝚜(word)))

val_dataset['kr_indices'] = kr_indices

In [12]:
en_indices = []
for word in test_dataset['en_tokens']:
  en_indices.append(torch.LongTensor(𝚎𝚗_𝚟𝚘𝚌𝚊𝚋.𝚕𝚘𝚘𝚔𝚞𝚙_𝚒𝚗𝚍𝚒𝚌𝚎𝚜(word)))

test_dataset['en_indices'] = en_indices

kr_indices = []
for word in test_dataset['kr_tokens']:
  kr_indices.append(torch.LongTensor(𝚔𝚛_𝚟𝚘𝚌𝚊𝚋.𝚕𝚘𝚘𝚔𝚞𝚙_𝚒𝚗𝚍𝚒𝚌𝚎𝚜(word)))

test_dataset['kr_indices'] = kr_indices

# Data Loaders

In [13]:
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        item = self.dataframe.iloc[idx]
        en_indices = item['en_indices']
        kr_indices = item['kr_indices']
        return en_indices, kr_indices

In [14]:
batch_no = 80

def collate_fn(batch):
    en_indices, kr_indices = zip(*batch)
    en_indices_padded = pad_sequence(en_indices, batch_first=True, padding_value=0)
    kr_indices_padded = pad_sequence(kr_indices, batch_first=True, padding_value=0)

    return en_indices_padded, kr_indices_padded

train_custom_dataset = CustomDataset(train_dataset[['en_indices', 'kr_indices']])
val_custom_dataset = CustomDataset(val_dataset[['en_indices', 'kr_indices']])
test_custom_dataset = CustomDataset(test_dataset[['en_indices', 'kr_indices']])

train_loader = DataLoader(train_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)

In [15]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_indices"] for example in batch]
        batch_de_ids = [example["de_indices"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    return collate_fn

def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

### 잘 매칭이 되었는지 확인해보기!

In [16]:
for i, (en_indices, kr_indices) in enumerate(train_loader):
    print(f"Batch {i+1}")
    print(f"Shape of en_indices: {en_indices.shape}")
    print(f"Shape of kr_indices: {kr_indices.shape}")

    print("Sample en_indices batch:", en_indices[:1])
    print("Sample kr_indices batch:", kr_indices[:1])

    if i == 1:
        break

Batch 1
Shape of en_indices: torch.Size([80, 31])
Shape of kr_indices: torch.Size([80, 40])
Sample en_indices batch: tensor([[   2,   15,   87,    7, 8333, 1033,   27,   15,  209,   55,   65,  132,
          827, 1075,   55, 6599, 1032,   76,   31,   25,  473,   42,    5,  842,
           21, 1551,    4,    3,    0,    0,    0]])
Sample kr_indices batch: tensor([[   2, 6329,   17, 2077,   28,  287,   12,   58,   11,  349,   13, 3589,
            9,   39, 2820,    5,   21,  324,  157,  159,  966,    8,  407,   81,
            5,   12,   58,   11,   51, 1397,   20,   47,   12,   58,   11, 6329,
            7,   18,    4,    3]])
Batch 2
Shape of en_indices: torch.Size([80, 31])
Shape of kr_indices: torch.Size([80, 32])
Sample en_indices batch: tensor([[  2,  13,   8, 106,   0,   6, 316,  17, 588, 925,  41, 144, 285, 226,
          11,  88, 498,   4,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0]])
Sample kr_indices batch: tensor([[   2,  918,    6,  768,   66, 

# 문제 3 (Modules without Attention!)

기본적인 Seq2seq 모듈들은 Teacher Forcing 까지 포함해서 진행해주시길 바랍니다.

In [17]:
class Encoder(nn.Module):
  """
  Encoder 내에서는 Input Language 의 Vocab 크기, Embedding 크기, Hidden 크기가 들어갑니다.

  Embedding 과정에서 Input 크기 --> Embedding 크기 로 진행됩니다.
  GRU 내에서는 Embedding 크기 --> Hidden 크기 로 진행됩니다.

  Input : [Batch_size, Input Sequences Max Size]
  Output : [Batch_size, Input Sequences Max Size, Hidden Size]
  Hidden : [1, Batch_size, Hidden Size]

  힌트 :
  - Batch 가 앞에 있기 때문에 GRU 는 batch_first = True 라는 파라미터를 설정해줘야 합니다.
  - GRU 에서는 Output, Hidden Vector, 총 2개를 뱉어줍니다.
  - GRU 는 nn.GRU, Embedder 는 nn.Embedding 를 사용해주면 됩니다.
  """
  def __init__(self, input_size, embedding_encoder_size, hidden_size):
    super().__init__()
    self.hid_size = hidden_size

    self.embedder = nn.Embedding(num_embeddings = input_size,
                                  embedding_dim= embedding_encoder_size)
    self.gru = nn.GRU(input_size = embedding_encoder_size,
                      hidden_size = hidden_size,
                      batch_first = True)

  def forward(self, input):
    embedded = self.embedder(input)

    output, hidden = self.gru(embedded)

    return output, hidden

In [18]:
class Decoder(nn.Module):
  """
  Decoder 내에서는 Hidden 크기, Embedding Decoder 크기 (Embedding Encoder 크기와 동일), Output Language 의 Vocab 의 크기가 들어갑니다.

  Embedding 과정에서 Output 크기 --> Embedding 크기 로 진행됩니다.
  GRU 내에서는 Embedding 크기 --> Hidden 크기 로 진행됩니다.
  Linear 내에서는 Hidden 크기 --> Output 크기 로 진행됩니다.

  Teacher Forcing 은 디코더를 구성하는 코드에서는 고려를 안해도 되며, 나중에 Seq2Seq 모듈에서 진행해주시길 바랍니다.

  Input 과 Hidden 크기의 경우 Seq2Seq 모듈 내에서 차원을 맞춰주시면 됩니다.
  """
  def __init__(self, hidden_size, embedding_decoder_size, output_size):
    super().__init__()
    self.output_size = output_size
    self.hid_size = hidden_size

    self.embedder = nn.Embedding(num_embeddings = output_size,
                                 embedding_dim = embedding_decoder_size)
    self.gru = nn.GRU(input_size = embedding_decoder_size,
                      hidden_size = hidden_size,
                      batch_first = True)
    self.linear = nn.Linear(in_features = hidden_size,
                           out_features = output_size)
    self.softmax = nn.LogSoftmax(dim = 1)

  def forward(self, input, hidden):
    embedded = self.embedder(input.unsqueeze(1))
    embedded = F.relu(embedded)

    output, hidden = self.gru(embedded, hidden)

    preds = self.linear(output.squeeze(1))
    preds = self.softmax(preds)

    return preds, hidden

In [19]:
class Seq2Seq(nn.Module):
  """
  Seq2Seq 내에서는 Encoder, Decoder, Device 가 들어갑니다.

  Init 단계에서는 정말로 할 것이 크게 없으며, forward 단계에서 for loop 과 if 조건문들을 활용해서 Teacher Forcing 을 적용시켜주면 됩니다.

  Input : [Batch_size, Input Sequences Max Size]
  Target : [Batch_size, Target Sequences Max Size]
  Output : [Batch_size, Input Sequences Max Size, Input Language Vocab Size]

  힌트 :
  - outputs 란 0 으로 채워진 텐서를 하나 만들며 디코더의 t-번째 아웃풀을 outputs 의 t-번째 칸에 넣으면서 for loop 을 진행해주시면 됩니다.
  - Target 의 첫번째 요소 (:, 0) 는 항상 <sos> 입니다! 이것을 디코더의 첫 Input 으로 넣어준 다음에 for loop 을 진행해주시면 됩니다.
  - random.random() < teacher_forcing_ratio 를 통해서 조건문을 만들 수가 있습니다!
  - Teacher Force 가 적용이 안될 때는디코더의 아웃풋에는 모든 토큰들에 대한 점수가 나오게 됩니다.
    이것들 중에서 가장 최댓값만 중요하기에 argmax 를 이용해서 최댓값의 인덱스를 다음 Decoder 의 Input 으로 넣어주면 됩니다.
  - squeeze 와 unsqueeze 를 적극적으로 활용해서 차원을 조정해주시면 됩니다!
  """
  def __init__(self, encoder, decoder, device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device

  def forward(self, input, target, teacher_forcing_ratio = 0.5):
    batch_size = target.shape[0]
    trg_len = target.shape[1] # length of target token
    trg_vocab_size = self.decoder.output_size # dim of context vector

    # tensor for saving decoder's output
    outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)

    # initial hidden state
    encoder_output, hidden = self.encoder(input)

    input = target[:, 0] # always <sos>

    for target_num in range(1, trg_len):

        preds, hidden = self.decoder(input, hidden)

        # saving prediction
        outputs[:, target_num] = preds

        # teacher forcing
        teacher_force = random.random() < teacher_forcing_ratio

        top1 = preds.argmax(dim = 1)

        input = target[:, target_num] if teacher_force else top1

    return outputs

# Training

In [20]:
input_dim = len(en_vocab)
output_dim = len(kr_vocab)
encoder_embedding_dim = 512
decoder_embedding_dim = 512
hidden_dim = 1024

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

encoder = Encoder(input_dim, encoder_embedding_dim, hidden_dim)
decoder = Decoder(hidden_dim, decoder_embedding_dim, output_dim)
model = Seq2Seq(encoder, decoder, device).to(device)

Device: cuda


### 모델 확인 + Initialize 시키기

In [21]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 30,502,090 trainable parameters


### 하이퍼 파라미터 설정!

In [22]:
unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

n_epochs = 10
teacher_forcing_ratio = 0.5

## Training 코드!

In [23]:
for epoch in tqdm.notebook.tqdm(range(n_epochs)):
    model.train()
    epoch_loss = 0
    for (en_indices, kr_indices) in train_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        optimizer.zero_grad()

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio)  # Target 에서는 <eos> 는 무시하고 진행합니다.
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)

        kr_indices = kr_indices[:, 1:].contiguous().view(-1)  # Target 에서 <sos> 를 무시하기 위함입니다.

        loss = criterion(output, kr_indices)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    train_loss = epoch_loss

    model.eval()
    epoch_loss = 0
    with torch.no_grad():
      for (en_indices, kr_indices) in val_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio = 0.0)
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        kr_indices = kr_indices[:, 1:].contiguous().view(-1)

        loss = criterion(output, kr_indices)
        epoch_loss += loss.item()

    print(f'Epoch: {epoch+1}, Train Loss : {train_loss / len(train_loader)}, Val Loss: {epoch_loss / len(val_loader)}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1, Train Loss : 2.8781522013346352, Val Loss: 2.9785741510845365
Epoch: 2, Train Loss : 2.5311691446304323, Val Loss: 2.8170176952604264
Epoch: 3, Train Loss : 2.3440015796025593, Val Loss: 2.8152691636766707
Epoch: 4, Train Loss : 2.1745183674494424, Val Loss: 2.772850948666769
Epoch: 5, Train Loss : 2.0201312653223673, Val Loss: 2.7645150063529846
Epoch: 6, Train Loss : 1.873709085782369, Val Loss: 2.7806236479017468
Epoch: 7, Train Loss : 1.717460439046224, Val Loss: 2.8278894424438477
Epoch: 8, Train Loss : 1.5723229036331177, Val Loss: 2.8990423982105558
Epoch: 9, Train Loss : 1.4416243314743042, Val Loss: 2.9500746613457087
Epoch: 10, Train Loss : 1.335177798430125, Val Loss: 3.0779825543600414


파라미터 저장

In [24]:
path_to_save = os.path.join(path_to_folder, "model_without_attention.pt")
torch.save(model.state_dict(), path_to_save)

### 적용시켜보기

In [25]:
test_batch_en, test_batch_kr = next(iter(test_loader))
test_en = test_batch_en[0].numpy()
test_kr = test_batch_kr[0].numpy()
model.eval()
with torch.no_grad():
  output = model(test_batch_en.to(device), test_batch_kr.to(device)[:, :-1], teacher_forcing_ratio = 0.0)
  print(en_vocab.lookup_tokens(test_en))
  print(kr_vocab.lookup_tokens(test_kr))
  print(kr_vocab.lookup_tokens(output.argmax(2).cpu().numpy()[0]))

['<sos>', 'then', 'you', 'have', "n't", 'been', 'to', 'the', 'stadium', 'in', 'incheon', ',', 'either', '?', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<sos>', '그런데', '인천', '에', '있', '는', '경기장', '도', '안', '가보', '았', '단', '말', '이', '에요', '?', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '근처', '에', '가시', '에', '가시', '는', '것', '가야', '이', '있', '나요', '?', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']


\<unk\> 는 저희 Vocabulary 에는 존재하지 않는 토큰들을 의미합니다.

아무리 epoch 를 늘리거나 무엇을 해보려고 해도 완전히 좋은 결과가 나오지는 않을 것입니다. 해당 이유는 Attention 이 적용이 안되었기 때문이며, 다음과 같이 Attention 을 적용시켜서 한번 진행하면 결과가 다르다는 것을 알 수가 있습니다.

# 문제 4 (Modules WITH Attention)

만약에 메모리가 다 찼다면 해당 Run 을 Reconnect 시켜서 위의 Output 만 남긴 채로 제출하시면 됩니다.

**이렇게 진행하게 된다면 당연히 문제 3 이전의 코드들은 다시 돌려야 됩니다!!**

In [ ]:
class Encoder(nn.Module):
  """
  이전과 Encoder 부분은 달라지는게 없습니다!
  """
  def __init__(self, input_size, embedding_encoder_size, hidden_size):
    super().__init__()
    self.hid_size = hidden_size

    self.embedder = nn.Embedding(num_embeddings = input_size,
                                  embedding_dim= embedding_encoder_size)
    self.gru = nn.GRU(input_size = embedding_encoder_size,
                      hidden_size = hidden_size,
                      batch_first = True)

  def forward(self, input):
    embedded = self.embedder(input)

    output, hidden = self.gru(embedded)

    return output, hidden

In [ ]:
class Attention(nn.Module):
  """
  해당 모듈에서는 Attention Value 까지 찾는 단계입니다.

  Encoder 내에서의 Output 들의 개수마다 for loop 을 진행시켜서 합산시켜서 Attention Value 를 구하는 방법도 있지만,
  Encoder Output 들을 모두 Matrix 으로 표현해서 구하는 방법도 있습니다.

  Decoder_Outputs : [Batch_size, 1, Hidden_size]
  Encoder_Outputs : [Batch_size, Sequence Length, Hidden_size]

  힌트 :
  - Decoder_Outputs 의 차원이 위와 같이 안 나온다면 차원 조정을 위해서 squeeze, unsqueeze, transpose 등의 함수들을 활용해주면 됩니다.
  - Matrix Multiplication 을 진행할 때에 Batch 까지 곱해지는 것을 방지하기 위해서 torch.bmm(x, y) 를 활용해주면 됩니다.
  - 곱해줄 때에는 그저 x * y 를 해주면 됩니다.
  - 합산할 때에는 배치도 고려해야 함으로 torch.sum(x, dim = 1) 을 활용해주면 됩니다.
  - 해당 모듈 내에서 필요한 nn. 함수는 오로지 Softmax 밖에 없습니다.
  """

  def __init__(self, hidden_size):
    super().__init__()
    self.hidden_size = hidden_size
    self.softmax = nn.Softmax(dim = 1)

  def forward(self, decoder_outputs, encoder_outputs):
    score = torch.bmm(encoder_outputs,
                      decoder_outputs.view(-1, self.hidden_size,1))
    att_dis = self.softmax(score) # attention distribution [Batch_size, Sequence Length, 1]
    attention_value = torch.sum(encoder_outputs * att_dis, dim = 1)

    return attention_value.unsqueeze(dim=1) # [Batch_size, Hidden_size, 1]

In [ ]:
class Decoder(nn.Module):
  """
  Attention Value 를 Attention 모듈을 통해서 구했기 때문에 Concatenate, Tangent Hyperbolic 등의 것들이 여기에서 적용이 됩니다.

  힌트 :
  - 모든 것을 진행하기 전에 앞서서 먼저 GRU 를 지나고 나서 Output 을 활용해야 합니다.
  - Attention 모듈도 사용해야 하기 때문에 해당 모듈을 현 모듈에서 정의도 해줘야 합니다.
  - Concatenate 의 경우 torch.cat((x, y), dim = 1) 형태로 사용하면 됩니다.
  - W_c 와 v_t 를 곱해줄 때 W_c 는 가중치 행렬이며, 이것은 그저 nn.Linear 형태로 표현이 가능합니다.
  """
  def __init__(self, hidden_size, embedding_decoder_size, output_size):
    super().__init__()
    self.hidden_size = hidden_size
    self.embedding_decoder_size = embedding_decoder_size
    self.output_size = output_size

    self.embedder = nn.Embedding(num_embeddings = output_size,
                                  embedding_dim = embedding_decoder_size)
    self.gru = nn.GRU(input_size = embedding_decoder_size,
                      hidden_size = hidden_size,
                      batch_first = True)

    self.attn_val = Attention(hidden_size)

    self.W_c = nn.Linear(in_features = hidden_size*2,
                          out_features = embedding_decoder_size)
    self.tanh = nn.Tanh()
    self.last_liner = nn.Linear(in_features = embedding_decoder_size,
                          out_features = output_size)
    self.softmax = nn.LogSoftmax(dim=2)


  def forward(self, input, hidden, encoder_outputs):
    embedded = self.embedder(input).unsqueeze(dim = 1)
    gru_out, gru_hidden = self.gru(embedded, hidden)
    attention_value = self.attn_val(gru_out.unsqueeze(dim = 1),
                                    encoder_outputs) # [Batch_size, 1, Hidden_size]
    con = torch.cat((attention_value, gru_out), dim = 2) # [Batch_size, 1, Hidden_size*2]
    out = self.W_c(con) # [Batch_size, 1, embedding_decoder_size]
    out = self.tanh(out)
    out = self.last_liner(out) # [Batch_size, 1, output_size]
    out = self.softmax(out)

    return out.squeeze(dim=1), gru_hidden # [Batch_size, output_size], [1, Batch_size, hidden_dim]

In [ ]:
class Seq2Seq(nn.Module):
  """
  이전과 Seq2Seq 부분은 크게 달라지는게 없습니다!
  단지 디코더가 Input 을 하나 더 받기 때문에 조심해주시길 바랍니다.
  """
  def __init__(self, encoder, decoder, device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device

  def forward(self, input, target, teacher_forcing_ratio = 0.5):

    batch_size = target.shape[0]
    trg_len = target.shape[1] # length of target token
    trg_vocab_size = self.decoder.output_size # dim of context vector

    # tensor for saving decoder's output
    outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)

    # initial hidden state
    encoder_outputs, hidden = self.encoder(input)
    # hidden = hidden.squeeze(dim = 1)

    input = target[:, 0] # always <sos>

    for target_num in range(1, trg_len):

        preds, hidden = self.decoder(input, hidden, encoder_outputs)

        # saving prediction
        outputs[:, target_num] = preds

        # teacher forcing
        teacher_force = random.random() < teacher_forcing_ratio

        top1 = preds.argmax(dim = 1)

        input = target[:, target_num] if teacher_force else top1

    return outputs

# Training

In [ ]:
input_dim = len(en_vocab)
output_dim = len(kr_vocab)
encoder_embedding_dim = 512
decoder_embedding_dim = 512
hidden_dim = 1024

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(input_dim, encoder_embedding_dim, hidden_dim)
decoder = Decoder(hidden_dim, decoder_embedding_dim, output_dim)
model = Seq2Seq(encoder, decoder, device).to(device)

### 모델 확인 + Initialize 시키기

In [ ]:
model.apply(init_weights)
print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 26,204,874 trainable parameters


### 하이퍼 파라미터 설정!

In [ ]:
unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

n_epochs = 10
teacher_forcing_ratio = 0.5

### 학습 시키기!

In [ ]:
for epoch in tqdm.notebook.tqdm(range(n_epochs)):
    model.train()
    epoch_loss = 0
    for (en_indices, kr_indices) in train_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        optimizer.zero_grad()

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio)  # Target 에서는 <eos> 는 무시하고 진행합니다.
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)

        kr_indices = kr_indices[:, 1:].contiguous().view(-1)  # Target 에서 <sos> 를 무시하기 위함입니다.

        loss = criterion(output, kr_indices)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    train_loss = epoch_loss
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
      for (en_indices, kr_indices) in val_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio = 0.0)
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        kr_indices = kr_indices[:, 1:].contiguous().view(-1)

        loss = criterion(output, kr_indices)
        epoch_loss += loss.item()
    print(f'Epoch: {epoch+1}, Train Loss : {train_loss / len(train_loader)}, Val Loss: {epoch_loss / len(val_loader)}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1, Train Loss : 2.905685092926025, Val Loss: 2.9005098305051287
Epoch: 2, Train Loss : 2.5444347162246705, Val Loss: 3.118109207304697
Epoch: 3, Train Loss : 2.3389099451700845, Val Loss: 2.8220976496499683
Epoch: 4, Train Loss : 2.1595525935490927, Val Loss: 2.879098517554147
Epoch: 5, Train Loss : 1.968269893328349, Val Loss: 2.700178725378854
Epoch: 6, Train Loss : 1.8038058045705159, Val Loss: 2.6719229902539934
Epoch: 7, Train Loss : 1.6481325976053873, Val Loss: 2.808993782315935
Epoch: 8, Train Loss : 1.4964544188181559, Val Loss: 2.7821810737488764
Epoch: 9, Train Loss : 1.388798588434855, Val Loss: 2.9022394589015414
Epoch: 10, Train Loss : 1.284762630780538, Val Loss: 2.9236826669602167


### 파라미터 저장

In [ ]:
path_to_save = os.path.join(path_to_folder, "model_WITH_attention.pt")
torch.save(model.state_dict(), path_to_save)

### 적용시켜보기

In [ ]:
test_batch_en, test_batch_kr = next(iter(train_loader))
test_en = test_batch_en[0].numpy()
test_kr = test_batch_kr[0].numpy()
model.eval()
with torch.no_grad():
  output = model(test_batch_en.to(device), test_batch_kr.to(device)[:, :-1], teacher_forcing_ratio = 0.0)
  print(en_vocab.lookup_tokens(test_en))
  print(kr_vocab.lookup_tokens(test_kr))
  print(kr_vocab.lookup_tokens(output.argmax(2).cpu().numpy()[0]))

['<sos>', 'it', 'makes', 'corals', 'into', '<unk>', 'by', 'eating', 'the', 'corals', "'", '<unk>', '.', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<sos>', '그것', '은', '산호', '의', '<unk>', '을', '먹', '어', '산호', '를', '화석', '으로', '만들', 'ㅂ니다', '.', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '은', '산호', '의', '산호', '의', '산호', '을', '산호', '으로', '산호', '으로', '만들', '만들', '.', 'ㅂ니다', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']


아마 결과가 온전히 잘 나오지는 않을 겁입니다. 해당 이유는 단순히 파라미터 개수가 적기 때문이며, 잘 나오기 위해서는 이보다 훨씬 늘려야 합니다. 원하신다면 Hidden_size, Embedding_size 를 늘려서 진행하는 것도 가능하긴 합니다!

과제 하시느라 정말로 고생하셨습니다!